# Custom Chatbot Project

I chose the 2023 fashion trends dataset in order to provide more specific information surrounding trends of that time period.

## Data Wrangling


In [1]:
import pandas as pd

#Load CSV data
df = pd.read_csv('data/2023_fashion_trends.csv')

In [2]:
#Extract text data columnn
df_cleaned = df["Trends"]

## Custom Query Completion


In [3]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))

    context = []
    #for text in get_rows_sorted_by_relevance(question, df)["text"].values:
    for text in df:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return current_token_count, prompt_template.format("\n\n###\n\n".join(context), question)

In [4]:
max_token_count = 4097
max_response_length = 1000
question = "What should I wear for a Y2K party?"

[num_tokens, prompt] = create_prompt(question, df_cleaned, max_token_count - max_response_length)

In [5]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "YOUR API KEY"

COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model

    If the model produces an error, return an empty string
    """

    [num_tokens, prompt] = create_prompt(question, df, max_prompt_tokens - max_answer_tokens)

    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return num_tokens, response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

In [6]:
[num_tokens,response] = answer_question(
    question, df_cleaned, max_prompt_tokens=4097, max_answer_tokens=1000
)

In [7]:
response

'You could wear a sheer cellophane-like dress over a boldly hued maxi skirt, a slip satin long skirt with a white ribbed tank and pointed toe kitten heels, or a denim maxi skirt with a corresponding denim corset top and some chunky platforms. You could also consider incorporating pastels, denim-on-denim, or mesh pieces, as these trends were popular in the Y2K era. Ultimately, the fashion you choose to wear for a Y2K party should reflect your personal style and make you feel confident and stylish.'

## Custom Performance Demonstration


### Question 1

In [8]:
#ChatGPT without provided fasion 2023 context:
question1 = "It's 2023, is a Canadian tuxedo in or out?"

max_response_length = 150

response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=question1,
    max_tokens=max_response_length
)

print("Question:\n" + question1 + "\n")
print("Answer:\n" + response["choices"][0]["text"].strip())

Question:
It's 2023, is a Canadian tuxedo in or out?

Answer:
In 2023, fashion trends are constantly changing, so it is difficult to predict if a Canadian tuxedo will be in or out. However, fashion experts believe that denim-on-denim looks will continue to be popular in the upcoming years, so it's possible that Canadian tuxedos will still be considered stylish. Ultimately, fashion is a personal choice, so wearing a Canadian tuxedo should be based on individual preference rather than trends.


In [9]:
#ChatGPT with provided fasion 2023 context:
_ , prompt_text= create_prompt(question1, df_cleaned, max_token_count- max_response_length)

response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt_text,
    max_tokens=max_response_length
)

print("Question:\n" + question1 + "\n")
print("Answer:\n" + response["choices"][0]["text"].strip())

Question:
It's 2023, is a Canadian tuxedo in or out?

Answer:
In.


### Question 2

In [10]:
#ChatGPT without provided fasion 2023 context:
question2 = "What size bags are fashionable in 2023?"

response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=question2,
    max_tokens=max_response_length
)

print("Question:\n" + question2 + "\n")
print("Answer:\n" + response["choices"][0]["text"].strip())

Question:
What size bags are fashionable in 2023?

Answer:
It is difficult to predict the exact size of bags that will be popular in 2023. Fashion trends constantly change and evolve, and what is considered fashionable can vary greatly from year to year. However, some predictions for popular bag sizes in 2023 include:

1. Oversized bags: Large, roomy bags have been popular for a few years now and are likely to continue being trendy in 2023. These bags are not only fashionable, but also practical, as they allow you to carry all your essentials without having to sacrifice style.

2. Mini bags: On the other end of the spectrum, mini bags are also expected to remain popular in 2023. These tiny bags are perfect for occasions where you only need to carry


In [11]:
#ChatGPT with provided fasion 2023 context:
_ , prompt_text= create_prompt(question2, df_cleaned, max_token_count- max_response_length)

response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt_text,
    max_tokens=max_response_length
)

print("Question:\n" + question2 + "\n")
print("Answer:\n" + response["choices"][0]["text"].strip())

Question:
What size bags are fashionable in 2023?

Answer:
Oversized bags are fashionable in 2023, as seen on the runway and predicted by fashion experts and stylists.
